In [1]:
import numpy as np
from sklearn import preprocessing


In [6]:
raw_data=np.loadtxt('F:\\Audiobooks_data.csv',delimiter=',')
raw_data

array([[8.7300e+02, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [6.1100e+02, 1.4040e+03, 2.8080e+03, ..., 0.0000e+00, 1.8200e+02,
        1.0000e+00],
       [7.0500e+02, 3.2400e+02, 3.2400e+02, ..., 1.0000e+00, 3.3400e+02,
        1.0000e+00],
       ...,
       [2.8671e+04, 1.0800e+03, 1.0800e+03, ..., 0.0000e+00, 2.9000e+01,
        0.0000e+00],
       [3.1134e+04, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.2832e+04, 1.6200e+03, 1.6200e+03, ..., 0.0000e+00, 9.0000e+01,
        0.0000e+00]])

In [18]:
unscaled_inputs_all=raw_data[:,1:-1]                       # to  eliminate 1st and last column
unscaled_inputs_all.shape

(14084, 10)

In [11]:
targets_all=raw_data[:,-1]
targets_all

array([1., 1., 1., ..., 0., 0., 0.])

# balance datasets

In [19]:
num_one_target=int(np.sum(targets_all))
zero_target_counter=0
indices_to_remove=[]
for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_target_counter+=1
        if zero_target_counter>num_one_target:
            indices_to_remove.append(i)
            
unscaled_balanced_input=np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
targets_balanced=np.delete(targets_all,indices_to_remove,axis=0)
unscaled_balanced_input.shape

(4474, 10)

# standarized_inputs

In [21]:
scaled_inputs=preprocessing.scale(unscaled_balanced_input)
scaled_inputs

array([[ 1.18956512,  0.36398846,  0.67728889, ..., -0.8635056 ,
        -0.20536617, -0.77240946],
       [-0.33022754,  1.10843845, -0.08841391, ..., -0.8635056 ,
        -0.20536617,  1.16499791],
       [-2.50135991, -1.74528653,  0.67728889, ..., -0.8635056 ,
         2.23179102,  2.78305242],
       ...,
       [ 1.18956512,  0.36398846,  0.67728889, ..., -0.20129479,
        -0.20536617, -0.62337812],
       [ 1.18956512,  0.36398846,  0.27347444, ..., -0.20129479,
        -0.20536617,  0.21758442],
       [ 1.18956512,  0.36398846,  0.20727535, ..., -0.20129479,
        -0.20536617, -0.51692717]])

# shuffle the data

In [23]:
shuffled_indices=np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs=scaled_inputs[shuffled_indices]
shuffled_targets=targets_balanced[shuffled_indices]

# split_dataseṭ into_train_and_split

In [30]:
sample_size=int(shuffled_inputs.shape[0])
train_size=int(0.8*sample_size)
validation_size=int(0.1*sample_size)
test_size=sample_size-train_size-validation_size

train_inputs=shuffled_inputs[:train_size]
train_target=shuffled_targets[:train_size]

validation_inputs=shuffled_inputs[train_size:train_size+validation_size]
validation_targets=shuffled_targets[train_size:train_size+validation_size]

test_inputs=shuffled_inputs[-test_size:]
test_targets=shuffled_targets[-test_size:]



In [35]:
# to check the data balance

print(train_size, np.sum(train_target)/train_size)
print(validation_size, np.sum(validation_targets)/validation_size)
print(test_size,np.sum(test_targets)/test_size)

3579 0.5051690416317407
447 0.4720357941834452
448 0.48660714285714285


this_shows_that data is equally distributed in all samples

# save_data into train_test_validation

In [36]:
np.savez('Audio data train',inputs=train_inputs,targets=train_target)
np.savez('Audio data validation',inputs=validation_inputs,targets=validation_targets)
np.savez('Audio data test',inputs=test_inputs,targets=test_targets)

In [4]:
import tensorflow as tf
import numpy as np

In [5]:
npz=np.load('Audio data train.npz')
train_inputs=npz['inputs'].astype(np.float)
train_target=npz['targets'].astype(np.int)

npz=np.load('Audio data validation.npz')
validation_inputs=npz['inputs'].astype(np.float)
validation_target=npz['targets'].astype(np.int)

npz=np.load('Audio data test.npz')
test_inputs=npz['inputs'].astype(np.float)
test_target=npz['targets'].astype(np.int)



model building

In [14]:
input_size=10
output_size=2
hidden_layer_size=50

model=tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='softmax')
                          ])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

batch_size=100
max_epoch=100
model.fit(train_inputs,train_target,batch_size=batch_size,epochs=max_epoch,
          validation_data=(validation_inputs,validation_target),verbose=2)

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 2s - loss: 3.0942 - accuracy: 0.4602 - val_loss: 2.2040 - val_accuracy: 0.6040
Epoch 2/100
3579/3579 - 0s - loss: 1.2477 - accuracy: 0.6675 - val_loss: 0.6412 - val_accuracy: 0.8255
Epoch 3/100
3579/3579 - 0s - loss: 0.4570 - accuracy: 0.8664 - val_loss: 0.4173 - val_accuracy: 0.8523
Epoch 4/100
3579/3579 - 0s - loss: 0.3584 - accuracy: 0.8782 - val_loss: 0.3803 - val_accuracy: 0.8568
Epoch 5/100
3579/3579 - 0s - loss: 0.3314 - accuracy: 0.8832 - val_loss: 0.3621 - val_accuracy: 0.8613
Epoch 6/100
3579/3579 - 0s - loss: 0.3174 - accuracy: 0.8885 - val_loss: 0.3522 - val_accuracy: 0.8658
Epoch 7/100
3579/3579 - 0s - loss: 0.3065 - accuracy: 0.8863 - val_loss: 0.3453 - val_accuracy: 0.8680
Epoch 8/100
3579/3579 - 0s - loss: 0.2966 - accuracy: 0.8916 - val_loss: 0.3186 - val_accuracy: 0.8792
Epoch 9/100
3579/3579 - 0s - loss: 0.2880 - accuracy: 0.8944 - val_loss: 0.3087 - val_accuracy: 0.8859
Epoch 10/100
3579/3579 - 0

Epoch 80/100
3579/3579 - 0s - loss: 0.2112 - accuracy: 0.9198 - val_loss: 0.2604 - val_accuracy: 0.9038
Epoch 81/100
3579/3579 - 0s - loss: 0.2108 - accuracy: 0.9195 - val_loss: 0.2515 - val_accuracy: 0.9083
Epoch 82/100
3579/3579 - 0s - loss: 0.2114 - accuracy: 0.9204 - val_loss: 0.2558 - val_accuracy: 0.8993
Epoch 83/100
3579/3579 - 0s - loss: 0.2120 - accuracy: 0.9209 - val_loss: 0.2592 - val_accuracy: 0.9038
Epoch 84/100
3579/3579 - 0s - loss: 0.2100 - accuracy: 0.9193 - val_loss: 0.2469 - val_accuracy: 0.9060
Epoch 85/100
3579/3579 - 0s - loss: 0.2114 - accuracy: 0.9193 - val_loss: 0.2600 - val_accuracy: 0.9083
Epoch 86/100
3579/3579 - 0s - loss: 0.2098 - accuracy: 0.9195 - val_loss: 0.2604 - val_accuracy: 0.9060
Epoch 87/100
3579/3579 - 0s - loss: 0.2117 - accuracy: 0.9193 - val_loss: 0.2773 - val_accuracy: 0.9016
Epoch 88/100
3579/3579 - 0s - loss: 0.2100 - accuracy: 0.9193 - val_loss: 0.2589 - val_accuracy: 0.8993
Epoch 89/100
3579/3579 - 0s - loss: 0.2099 - accuracy: 0.9201 - 

its seems like data is overfitted

# early_stopping_mechanism by using callback

In [19]:
input_size=10
output_size=2
hidden_layer_size=50

model=tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='softmax')
                          ])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

batch_size=100
max_epoch=100

early_stopping=tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_inputs,train_target,batch_size=batch_size,epochs=max_epoch,callbacks=[early_stopping],
          validation_data=(validation_inputs,validation_target),verbose=2)

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 1s - loss: 3.1557 - accuracy: 0.4876 - val_loss: 2.2040 - val_accuracy: 0.6555
Epoch 2/100
3579/3579 - 0s - loss: 1.1868 - accuracy: 0.7156 - val_loss: 0.5602 - val_accuracy: 0.8434
Epoch 3/100
3579/3579 - 0s - loss: 0.4247 - accuracy: 0.8690 - val_loss: 0.3951 - val_accuracy: 0.8568
Epoch 4/100
3579/3579 - 0s - loss: 0.3442 - accuracy: 0.8813 - val_loss: 0.3585 - val_accuracy: 0.8658
Epoch 5/100
3579/3579 - 0s - loss: 0.3206 - accuracy: 0.8863 - val_loss: 0.3442 - val_accuracy: 0.8680
Epoch 6/100
3579/3579 - 0s - loss: 0.3076 - accuracy: 0.8899 - val_loss: 0.3341 - val_accuracy: 0.8702
Epoch 7/100
3579/3579 - 0s - loss: 0.2972 - accuracy: 0.8905 - val_loss: 0.3218 - val_accuracy: 0.8792
Epoch 8/100
3579/3579 - 0s - loss: 0.2882 - accuracy: 0.8941 - val_loss: 0.3230 - val_accuracy: 0.8814
Epoch 9/100
3579/3579 - 0s - loss: 0.2815 - accuracy: 0.8966 - val_loss: 0.3119 - val_accuracy: 0.8837
Epoch 10/100
3579/3579 - 0

In [21]:
test_loss,test_accuracy=model.evaluate(test_inputs,test_target) 

448/448 [==============================] - 0s 448us/sample - loss: 0.2731 - accuracy: 0.9018
